# Generierung von Aufgaben zum Embedding: 
Wir verwenden den Quora scrape um verschiedene Topics zu erhalten. Damit generieren wir uns jeweils 20 diverse Aufgaben. 

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [11]:
# from huggingface_hub import hf_hub_download
# hf_hub_download(repo_id="SebastianBodza/Quora_deutsch_ger_Pairs_RL_DPO", filename="output.jsonl", repo_type="dataset")
import pandas as pd
import random 
import urllib.parse

quora = pd.read_json("/home/bodza/.cache/huggingface/hub/datasets--SebastianBodza--Quora_deutsch_ger_Pairs_RL_DPO/snapshots/f81129b4c2a5453b6c037e0571a52b701da8f6b8/output.jsonl", orient="records", lines=True)

def sample_topics(n=5): 
    samples = random.sample(quora["topic"].unique().tolist(), n)
    samples = [urllib.parse.unquote(encoded_string) for encoded_string in samples]
    return samples

In [49]:
prompt = """Create a list of potentially useful text retrieval tasks (RAG).
Stick to the following guidelines:
- Specify what the query is and what the requested documents are.
- Each retrieval task should cover a wide range of requests and should not be too specific.

Your output should always be just a list of strings, with about 5 elements each, and each element corresponds to a unique retrieval task in a set. Don't explain yourself or give anything else away. Be creative.

Create these tasks in the following areas:
{tasks}
Create all entries completely in German! Never use English! Never use "Sie"! Never use the german word "Sie"!"""

response_template ="'{category_1}':\n1."



In [57]:
from transformers import AutoTokenizer
import torch 
from auto_gptq import AutoGPTQForCausalLM


model_name = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"
revision="gptq-4bit-128g-actorder_True"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model_q = AutoGPTQForCausalLM.from_quantized(
    model_name,
    revision=revision,
    device="auto",
    use_triton=False,
    inject_fused_attention=False,
    inject_fused_mlp=False,
    disable_exllama=True,
    disable_exllamav2=True,
    use_marlin=True,
    torch_dtype=torch.float16
)



for n in range(5000):
    topics = sample_topics()
    formatted_prompt = tokenizer.apply_chat_template(conversation=[{"role": "user", "content":prompt.format(tasks="\n".join(topics))},{"role": "assistant", "content":response_template.format(category_1=topics[0])}], tokenize=False)
    formatted_prompt = formatted_prompt.removesuffix("</s>")
    print(formatted_prompt)

    if n%10 == 0: 


<s>[INST] Create a list of potentially useful text retrieval tasks (RAG).
Stick to the following guidelines:
- Specify what the query is and what the requested documents are.
- Each retrieval task should cover a wide range of requests and should not be too specific.

Your output should always be just a list of strings, with about 5 elements each, and each element corresponds to a unique retrieval task in a set. Don't explain yourself or give anything else away. Be creative.

Create these tasks in the following areas:
Eier-Nahrungsmittel
Amerikanischer-Konservatismus
Melbourne
Quanten-Chromodynamik
Selbständige-Unternehmer
Create all entries completely in German! Never use English! Never use "Sie"! Never use the german word "Sie"! [/INST]'Eier-Nahrungsmittel':
1.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision=revision)




In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer
device = torch.device("cuda:0")

model_q = AutoGPTQForCausalLM.from_quantized(
    model_id,
    device="cuda:0",
    use_triton=False,
    inject_fused_attention=False,
    inject_fused_mlp=False,
    disable_exllama=disable_exllama,
    disable_exllamav2=disable_exllamav2,
    use_marlin=use_marlin,
    torch_dtype=torch.float16
)

